# Nath et al. (2009)

In which the GMPE of Nath et al. (2009) is implemented. First coefficient tables are reprocessed for cut & paste in to the .py source code. Then, key figures in the original paper are reproduced for validation. Finally, test vectors are produced for automatic code verification using unittest.

In [ ]:
%matplotlib inline
%load_ext autoreload
import os
import warnings
import pandas as pd
import numpy as np
import csv as csv
#import toolbox as tb
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
from scipy.constants import g
import importlib

from openquake.hazardlib import geo, mfd, tom, pmf, imt, const, gsim, tests
import openquake.hazardlib.source as src
import openquake.hazardlib.scalerel as msr
import openquake.hazardlib.site as sit

from openquake.hazardlib.gsim.sharma_2009 import SharmaEtAl2009

for item in gsim.get_available_gsims().values():
    importlib.import_module(item.__module__, item.__name__)
    try:
        importlib.import_module(item.__module__.replace('.gsim','.tests.gsim') + '_test', item.__name__ + 'TestCase')
        #print 'imported', item.__name__ + 'TestCase'
    except ImportError:
        #print 'from', item.__module__.replace('.gsim','.tests.gsim') + '_test', 'import', item.__name__ + 'TestCase failed ...'
        pass


In [ ]:
df_coeffs = pd.DataFrame.from_csv('Tables/Table2.csv', index_col=None)
df_coeffs

In [ ]:
df_coeffs2 = pd.DataFrame.from_csv('sharma coefficients.csv', index_col=None)
df_coeffs2

In [ ]:
# how peculiar that there is just one repeated row (except for sigma) ...
# no mention of coefficients being reused for 0.1 and 0.2 s in the paper ...
df_coeffs2.diff() == 0

In [ ]:
coefficients = ['b%d' for i in range(1,7)]
df_coeffs.plot(x='T', y=['b1', 'b2', 'b3', 'b5', 'b6'], figsize=(6, 8), grid=True)
plt.axhspan(-0.1, 0.1, color='0.5', alpha=0.5)
plt.savefig('Sharma_coefficients.pdf', dpi=300, bbox_inches='tight')

In [ ]:
with open('Table2.txt','w') as f:
    f.write(df_coeffs[['T', 'b1', 'b2', 'b3', 'b5', 'b6', 'sigma']].to_string(index=False))

In [ ]:
%autoreload 2
gmpe = SharmaEtAl2009()
print(type(gmpe).__name__)
print('Supported tectonic region: %s' 
      % gmpe.DEFINED_FOR_TECTONIC_REGION_TYPE)
print('Supported intensity measure types: %s' 
      % ', '.join([item.__name__ for item 
                   in gmpe.DEFINED_FOR_INTENSITY_MEASURE_TYPES]))
print('Supported component: %s' 
      % gmpe.DEFINED_FOR_INTENSITY_MEASURE_COMPONENT)
print('Supported standard deviations: %s' 
      % ', '.join([item for item              
                   in gmpe.DEFINED_FOR_STANDARD_DEVIATION_TYPES]))
print('Required site parameters: %s' 
      % ', '.join([item for item in gmpe.REQUIRES_SITES_PARAMETERS]))
print('Required rupture parameters: %s' 
      % ', '.join([item for item in gmpe.REQUIRES_RUPTURE_PARAMETERS]))
print('Required distance parameters: %s' 
      % ', '.join([item for item in gmpe.REQUIRES_DISTANCES]))

In [ ]:
def summarize(obj):
    print '\n'.join("%s: %s" % (item, getattr(obj, item))
                    for item in dir(obj) if item[0] != '_')
    
def isnumeric(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

In [ ]:
# fixed parameters
tectonic_region_type = 'Active Shallow Crust'
hypo_lon = 0 # degrees
hypo_lat = 0 # degrees
hypo_depth = 12.5 # km
uncertainty_types = [const.StdDev.TOTAL]

# set things up so that Rjb is epicentral distance
site_azimuth = 0 # degrees
hypo_strike = 90 # degrees
hypo_dip = 90 # degrees


In [ ]:
def compute_gmpe_overcomplicated(gmpe, mags, rakes, distances, vs30s, im_types):
    
    # set up sources
    mechanisms = [geo.NodalPlane(strike=hypo_strike, 
                                 dip=hypo_dip, rake=rake) 
                  for rake in rakes]
    prob = 1./len(rakes)
    nodal_plane_dist = pmf.PMF([(prob, mech) 
                                for mech in mechanisms])

    # derived parameters
    if np.array(mags).size > 1:
        delta_m = np.mean(np.diff(mags))
    else:
        delta_m = 1.
    mag_freq_dist = mfd.TruncatedGRMFD(
        min_mag=mags.min() - delta_m/2, 
        max_mag=mags.max() + delta_m/2, 
        bin_width=delta_m, 
        a_val=5.0, b_val=1.0),

    hypocenter = geo.Point(hypo_lon, hypo_lat, hypo_depth)

    source = src.PointSource(
        source_id='1',
        name='test',
        tectonic_region_type=tectonic_region_type,
        mfd=mag_freq_dist[0],
        rupture_mesh_spacing=2.,
        magnitude_scaling_relationship=msr.WC1994(),
        rupture_aspect_ratio=1.,
        temporal_occurrence_model=tom.PoissonTOM(50.),
        upper_seismogenic_depth=0.2*hypo_depth,
        lower_seismogenic_depth=2*hypo_depth,
        location=hypocenter,
        nodal_plane_distribution=nodal_plane_dist,
        hypocenter_distribution=pmf.PMF([(1, hypo_depth)]))

    # set up sites
    lons, lats = geo.geodetic.point_at(hypo_lon, hypo_lat, 
                                       site_azimuth, distances)
    site_locations = [geo.Point(lon, lat) for lon, lat in zip(lons, lats)]
    site_list = [sit.Site(location=loc, vs30=vs30, 
                    vs30measured=True, z1pt0=40., z2pt5=1.0) 
                 for vs30 in vs30s 
                 for loc in site_locations]
    site_collection = sit.SiteCollection(site_list)

    # compute ground motion
    context_maker = gsim.base.ContextMaker([gmpe])
    empty = True
    for rupture in source.iter_ruptures():

        # tweak required because ``min_mag`` and ``max_mag`` are rounded by 
        # TruncatedGRMFD so that  both are divisible by ``bin_width``
        rupture.mag = rupture.mag - delta_m/2.
        sctx, rctx, dctx = context_maker.make_contexts(
            site_collection, rupture)
        for i, im_type in enumerate(im_types):
            mean, [stddev] = gmpe.get_mean_and_stddevs(
                sctx, rctx, dctx, im_type, uncertainty_types)
            if i == 0:
                df_mean = pd.DataFrame({
                    'rup_mag': np.full_like(mean, rctx.mag),
                    'rup_rake': np.full_like(mean, rctx.rake),
                    'dist_rjb': dctx.rjb,
                    'site_vs30': sctx.vs30,
                    'damping': im_type.damping,
                        'S': (gmpe.get_site_type_dummy_variables(sctx)).astype('float'),
                        'H': np.full_like(mean, float(gmpe.get_fault_type_dummy_variables(rctx)))})
                df_stddev = df_mean.copy()
                df_mean['result_type'] = 'MEAN'
                df_stddev['result_type'] = 'TOTAL_STDDEV'
            df_mean[str(im_type.period)] = np.exp(mean)
            df_stddev[str(im_type.period)] = np.exp(stddev)
        if empty:
            df_means = df_mean
            df_stddevs = df_stddev
            empty = False
        else:
            df_means = pd.concat((df_means, df_mean), ignore_index=True)
            df_stddevs = pd.concat((df_stddevs, df_stddev), ignore_index=True)
    
    df_means.sort(['rup_rake','site_vs30','rup_mag','dist_rjb'], inplace=True)
    df_stddevs.sort(['rup_rake','site_vs30','rup_mag','dist_rjb'], inplace=True)
    df_means.index = range(len(df_means))
    df_stddevs.index = range(len(df_stddevs))
        
    #df_means = df_means.set_index(['rup_rake','site_vs30','rup_mag','dist_rjb']).sort()
    #df_stddevs = df_stddevs.set_index(['rup_rake','site_vs30','rup_mag','dist_rjb']).sort()

    return (df_means, df_stddevs)

In [ ]:
def compute_gmpe(gmpe, mags, rakes, distances, vs30s, im_types):
    
    rctx = gsim.base.RuptureContext()
    sctx = gsim.base.SitesContext()
    dctx = gsim.base.DistancesContext()

    empty = True
    for mag in mags:
        for rake in rakes:
            
            rctx.mag = mag
            rctx.rake = rake
            dctx.rjb = np.tile(distances, vs30s.size)
            sctx.vs30 = np.repeat(vs30s, distances.size)
            
            for i, im_type in enumerate(im_types):

                mean, [stddev] = gmpe.get_mean_and_stddevs(
                    sctx, rctx, dctx, im_type, uncertainty_types)
                                
                if i == 0:
                    df_mean = pd.DataFrame({
                        'rup_mag': np.full_like(mean, rctx.mag),
                        'rup_rake': np.full_like(mean, rctx.rake),
                        'dist_rjb': dctx.rjb,
                        'site_vs30': sctx.vs30,
                        'damping': im_type.damping,
                        'S': (gmpe.get_site_type_dummy_variables(sctx)).astype('float'),
                        'H': np.full_like(mean, float(gmpe.get_fault_type_dummy_variables(rctx)))})
                    df_stddev = df_mean.copy()
                    df_mean['result_type'] = 'MEAN'
                    df_stddev['result_type'] = 'TOTAL_STDDEV'
                df_mean[str(im_type.period)] = np.exp(mean)
                df_stddev[str(im_type.period)] = stddev
                
            if empty:
                df_means = df_mean
                df_stddevs = df_stddev
                empty = False
            else:
                df_means = pd.concat((df_means, df_mean), ignore_index=True)
                df_stddevs = pd.concat((df_stddevs, df_stddev), ignore_index=True)
    
    df_means.sort(['rup_rake','site_vs30','rup_mag','dist_rjb'], inplace=True)
    df_stddevs.sort(['rup_rake','site_vs30','rup_mag','dist_rjb'], inplace=True)
    df_means.index = range(len(df_means))
    df_stddevs.index = range(len(df_stddevs))
    
    #df_means = df_means.set_index(['rup_rake','site_vs30','rup_mag','dist_rjb']).sort()
    #df_stddevs = df_stddevs.set_index(['rup_rake','site_vs30','rup_mag','dist_rjb']).sort()

    return (df_means, df_stddevs)

In [ ]:
# generate data for Figure 6
mags = np.array([5., 6., 7.])
rakes = np.array([0., 90.]) # degrees
#distances = tb.logspace(1, 200, 6) # km
distances = np.array([1., 1.5, 2.2, 3.3, 4.7, 6.8, 
                      10., 15., 22., 33., 47., 68., 
                      100., 150.]) # km
vs30s = np.array([500., 2000.]) # m/s
im_types = [imt.SA(0.04, 5)]

df_means, df_stddevs = compute_gmpe(gmpe, mags, rakes, distances, vs30s, im_types)

pd.concat((df_means.head(), df_means.tail()))

In [ ]:
# generate data for Figures 7-9
mags = np.array([5., 6., 7.])
rakes = np.array([0., 90.]) # degrees
distances = np.array([10., 50., 100.]) # km
vs30s = np.array([500., 2000.]) # m/s
im_types = [imt.SA(T, 5) for T in df_coeffs['T']]

df_means2, df_stddevs2 = compute_gmpe(gmpe, mags, rakes, distances, vs30s, im_types)
pd.concat((df_means2.head(), df_means2.tail()))

In [ ]:
pd.concat((df_stddevs2.head(), df_stddevs2.tail()))

In [ ]:
# check that warning is raised for normal faulting
rctx = gsim.base.RuptureContext()
sctx = gsim.base.SitesContext()
dctx = gsim.base.DistancesContext()

rctx.mag = np.array([6.])
rctx.rake = np.array([-90.])
dctx.rjb = np.array([100.])
sctx.vs30 = np.array([1000.])

with warnings.catch_warnings(record=True) as w:
    # Cause all warnings to always be triggered.
    warnings.simplefilter('always')
    # Trigger a warning.
    mean, [stddev] = gmpe.get_mean_and_stddevs(
        sctx, rctx, dctx, im_types[0], uncertainty_types)
    # Verify some things
    assert len(w) == 1
    assert issubclass(w[-1].category, UserWarning)
    assert 'normal' in str(w[-1].message).lower()

print mean, '+/-', stddev


In [ ]:
# produce Figure 6
digitized_template = 'digitized/M%d_S%d_H%d_%gs.csv'

Ss = sorted(list(set(df_means['S'])))
Hs = sorted(list(set(df_means['H'])))
Rjbs = sorted(list(set(df_means['dist_rjb'])))
fig, axes = plt.subplots(len(Ss), len(Hs), figsize=(8,6),
                         sharex=True, sharey=True)
plt.subplots_adjust(wspace=0.075, hspace=0.1)
for i, S in enumerate(Ss):
    for j, H in enumerate(Hs):
        fig.sca(axes[i,j])
        df_axes = df_means[
            (df_means['S'] == S) &
            (df_means['H'] == H)]
        axis_label = 'S=%g\nH=%g' % (S, H)
        fig.gca().add_artist(AnchoredText(axis_label, loc=1, frameon=False))

        for mag in set(df_axes['rup_mag']):
            df_trace = df_axes[df_axes['rup_mag'] == mag]
                
            trace_label = 'M=%g' % mag
            h = plt.loglog(df_trace['dist_rjb'], df_trace[str(im_types[0].period)]*g, 
                       label=trace_label, alpha=0.5)
            
            digitized_file = digitized_template % (mag, S, H, im_types[0].period)
            if os.path.exists(digitized_file):
                data = np.genfromtxt(digitized_file, delimiter=',')
                plt.plot(data[:,0], data[:,1], 
                         color=h[0].get_color(), marker='x', 
                         linestyle='none', alpha=0.5)
            else:
                print '%s not available' % digitized_file
                
for ax in axes:
    ax[0].set_xlim((min(Rjbs), max(Rjbs)))
    ax[0].set_ylim((0.1, 5))
for ax in axes[1,:]:
    ax.set_xlabel('Joyner-Boore Distance [km]')
for ax in axes[:,0]:
    ax.set_ylabel('SA(T=%g s, $\\xi$=%g%%) [m/s$^2$]' 
          % (im_types[0].period, im_types[0].damping))
axes[1,0].legend(loc='lower left', labelspacing=0, fontsize=10, frameon=False)
plt.savefig('Figure_6_computed.pdf', dpi=300, bbox_inches='tight')

In [ ]:
# produce Figures 7-9
digitized_template = 'digitized/M%d_S%d_H%d_%gkm.csv'

Ss = sorted(list(set(df_means2['S'])))
Hs = sorted(list(set(df_means2['H'])))
Rjbs = sorted(list(set(df_means2['dist_rjb'])))
fig, axes = plt.subplots(len(Rjbs), 1, figsize=(6,10), sharex=True)
plt.subplots_adjust(hspace=0.1)
for i, dist in enumerate(Rjbs):
    
    fig.sca(axes[i])
    df_axes = df_means2[df_means2['dist_rjb'] == dist]
    axis_label = '%g km' % dist
    fig.gca().add_artist(AnchoredText(axis_label, loc=2, frameon=False))
    
    S = 0
    for H in Hs:
        for mag in set(df_axes['rup_mag']):
            df_trace = df_axes[
                (df_axes['rup_mag'] == mag) &
                (df_axes['S'] == S) & 
                (df_axes['H'] == H)]
                
            trace_label = 'M=%g, S=%d, H=%d' % (mag, S, H)
            if df_trace.size == 0:
                print 'No data found for', trace_label
            
            data_trace = df_trace.loc[:,np.array(map(isnumeric, df_trace.keys()))].T
            T = [float(item) for item in data_trace.index]
            SA = data_trace.values*g
            h = plt.plot(T, SA, label=trace_label, alpha=0.5)
            
            digitized_file = digitized_template % (mag, S, H, dist)
            if os.path.exists(digitized_file):
                data = np.genfromtxt(digitized_file, delimiter=',')
                plt.plot(data[:,0], data[:,1], 
                         color=h[0].get_color(), marker='x', 
                         linestyle='none', alpha=0.5)

            else:
                print '%s not available' % digitized_file
                
for ax in axes:
    ax.set_xlim((0, max(T)))
    ax.set_ylim((0, ax.get_ylim()[1]))
axes[-1].set_xlabel('Period [s]')
for ax in axes:
    ax.set_ylabel('SA($\\xi$=%g%%) [m/s$^2$]' % (im_types[0].damping))
axes[0].legend(loc='upper right', labelspacing=0, fontsize=10, frameon=False)
plt.savefig('Figures_7-9_computed.pdf', dpi=300, bbox_inches='tight')

In [ ]:
# produce plot of estimated residuals
Ss = sorted(list(set(df_stddevs2['S'])))
Hs = sorted(list(set(df_stddevs2['H'])))
Rjbs = sorted(list(set(df_stddevs2['dist_rjb'])))
fig, ax = plt.subplots(1, 1)

dist = Rjbs[0]   
df_axes = df_stddevs2[df_stddevs2['dist_rjb'] == dist]

S = Ss[0]
H = Hs[0]
mag = sorted(list(set(df_axes['rup_mag'])))[0]
df_trace = df_axes[
    (df_axes['rup_mag'] == mag) &
    (df_axes['S'] == S) & 
    (df_axes['H'] == H)]

trace_label = 'M=%g, S=%d, H=%d' % (mag, S, H)
if df_trace.size == 0:
    print 'No data found for', trace_label

data_trace = df_trace.loc[:,np.array(map(isnumeric, df_trace.keys()))].T
T = [float(item) for item in data_trace.index]
sigma = data_trace.values
plt.plot(T, sigma, label=trace_label, alpha=0.5)
                            
ax.set_xlim((0, max(T)))
ax.set_ylim((0, ax.get_ylim()[1]))
ax.set_xlabel('Period [s]')
ax.set_ylabel('Total standard deviation $\sigma$')
plt.savefig('Sigma_computed.pdf', dpi=300, bbox_inches='tight')

In [ ]:
# produce result file for unittest
test_path = '/home/nick/src/python/GEM/oq-hazardlib/openquake/hazardlib/tests/gsim/data/SDBK09/'
means_file = os.path.join(test_path, 'SDBK09_MEAN.csv')
stddev_file = os.path.join(test_path, 'SDBK09_STD_TOTAL.csv')
#df_means2.drop(['H','S'], axis=1).to_csv(means_file, index=False)
df_stddevs2.drop(['H','S'], axis=1).to_csv(stddev_file, index=False)

In [ ]:
# load dataset provided by authors
ref_file = os.path.join(test_path, 'SDBK09_MEAN.csv')
df_ref = pd.read_csv(ref_file, na_values='nan')

df_ref.sort(['rup_rake','site_vs30','rup_mag','dist_rjb'], inplace=True)
df_ref.index = range(len(df_ref))

mags = np.sort(np.array(list(set(df_ref['rup_mag']))))
rakes = np.sort(np.array(list(set(df_ref['rup_rake']))))
#rakes = rakes[rakes >= 0]
Rjbs = np.sort(np.array(list(set(df_ref['dist_rjb']))))
vs30s = np.sort(np.array(list(set(df_ref['site_vs30']))))
Ts = np.sort(np.array([float(item) for item in 
     df_ref.loc[:,np.array(map(isnumeric, df_ref.keys()))]]))
im_types = [imt.SA(T) for T in Ts]

print mags, rakes, Rjbs, vs30s
print Ts

In [ ]:
df_new, _ = compute_gmpe(gmpe, mags, rakes, Rjbs, vs30s, im_types)
df_new = df_new.drop(['H','S'], axis=1)
df_ref = df_ref[df_new.columns]

In [ ]:
df_ref.head()

In [ ]:
df_new.head()

In [ ]:
n_traces = len(rakes)*len(vs30s)*len(mags)
color_cycle = plt.cm.jet(np.linspace(0, 1, n_traces))

fig, axes = plt.subplots(len(Rjbs), 1, figsize=(6,15), sharex=True)
plt.subplots_adjust(hspace=0.1)
for i, dist in enumerate(Rjbs):
    
    fig.sca(axes[i])
    axis_label = 'rjb=%g' % dist
    fig.gca().add_artist(AnchoredText(axis_label, loc=1, frameon=False))
    fig.gca().set_color_cycle(color_cycle)
   
    for rake in rakes:
        for vs30 in vs30s:
            for mag in mags:
                trace_label = 'mag=%g, rake=%g, vs30=%g' % (mag, rake, vs30)
                
                df_trace = df_ref[
                    (df_ref['dist_rjb'] == dist) &
                    (df_ref['rup_mag'] == mag) &
                    (df_ref['rup_rake'] == rake) & 
                    (df_ref['site_vs30'] == vs30)]

                if df_trace.size == 0:
                    print 'No reference found for', trace_label

                data_trace = df_trace.loc[:,np.array(map(isnumeric, df_trace.keys()))].T
                SA = np.reshape(data_trace.values.T, (-1,))*g
                h = plt.plot(Ts, SA, label=trace_label, alpha=0.5)
                
                df_trace2 = df_new[
                    (df_new['dist_rjb'] == dist) &
                    (df_new['rup_mag'] == mag) &
                    (df_new['rup_rake'] == rake) & 
                    (df_new['site_vs30'] == vs30)]
                
                if df_trace2.size == 0:
                    print 'No new value found for', trace_label

                data_trace2 = df_trace2.loc[:,np.array(map(isnumeric, df_trace2.keys()))].T
                SA2 = np.reshape(data_trace2.values.T, (-1,))*g
                plt.plot(Ts, SA2, linestyle='none', marker='x', 
                         color=h[0].get_color(), alpha=0.5)
                
for ax in axes:
    ax.set_xlim((0, max(Ts)))
    ax.set_ylim((0, ax.get_ylim()[1]))
axes[-1].set_xlabel('Period [s]')
for ax in axes:
    ax.set_ylabel('SA($\\xi$=%g%%) [m/s$^2$]' % (im_types[0].damping))
axes[0].legend(loc='upper left', labelspacing=0, fontsize=10, bbox_to_anchor=(1, 1))
plt.savefig('Reference_result.png', dpi=300, bbox_inches='tight')

In [ ]:
def compare(df_new, df_ref):
    numeric = np.array([[isnumeric(item) for item in row[1]] for row in df_ref.iterrows()])
    non_zero = (df_ref != 0) & numeric
    df_dif = df_ref.copy()
    df_dif = df_dif.where(~numeric, df_new.where(numeric) - df_ref.where(numeric))
    df_dif = df_dif.where(~non_zero, df_dif.where(non_zero)/df_ref.where(non_zero))
    df_dif = df_dif.where(numeric, df_new.where(~numeric) != df_ref.where(~numeric))
    
    return df_dif

In [ ]:
# summarize the largest discrepancies
threshhold = 1e-8

ne_stacked = (compare(df_new, df_ref) > threshhold).stack()
changed = ne_stacked[ne_stacked]
changed.index.names = ['index', 'column']

difference_locations = np.where(compare(df_new, df_ref) > threshhold)
changed_from = df_new.values[difference_locations]
changed_to = df_ref.values[difference_locations]
pd.DataFrame({'from': changed_from, 'factor': changed_from/changed_to}, index=changed.index)

In [ ]:
g

In [ ]:
type(bool)